# Read GeoTIFF data through a BMI

This notebook describes how to open and read data from GeoTIFF files
using a [Basic Model Interface](https://bmi.readthedocs.io/) (BMI) through the `BmiGeoTiff` class included in the `bmi-geotiff` package.

## Setup

To ensure all dependencies are met, set up a conda environment using the environment file found in the root directory of this repository:
```
conda env create --file environment.yml
```

Then install the `bmi-geotiff` package:
```
pip install -e .
```

Import a set of libraries for later use:

In [ ]:
import numpy as np
from rasterio.plot import show

## Open a file

Import the `BmiGeoTiff` class from the `bmi-geotiff` package:

In [ ]:
from bmi_geotiff import BmiGeoTiff

Create an instance of this class.

In [ ]:
m = BmiGeoTiff()

Calling `help` on the instance displays all the BMI methods that are available.

In [ ]:
help(m)

The first step in using a BMI is calling the `initialize` method.
This method requires a configuration file that provides initial values for the `GeoTiff` library wrapped by the BMI.

A sample configuration file is provided in the **examples** directory.

In [ ]:
ls

In [ ]:
cat config.yaml

In this case, the configuration file simply lists the path to a GeoTIFF file
(here, the test file **RGB.byte.tif** from the [rasterio](https://rasterio.readthedocs.io/) project).

Call `initialize` with the sample configuration file:

In [ ]:
m.initialize("config.yaml")

The GeoTIFF file listed in the configuration file has now been opened,
and the information it contains can be accessed through BMI methods.

## Access data through the BMI

Now that we've opened the GeoTIFF file, let's access the data and metadata it contains through the BMI.
This will take a few steps.
It may seem cumbersome at first, but there's payoff at the end.

Start by displying the names of the variables exposed through the BMI.

In [ ]:
m.get_output_var_names()

The (long) names used for these variables are instances of [CSDMS Standard Names](https://csdms.colorado.edu/wiki/CSDMS_Standard_Names).
Standard Names are intended to be unambiguous; the tradeoff is that they tend to be long.
Here, the first variable is for the raster data stored in the file.

Find the data type of the raster.

In [ ]:
dtype = m.get_var_type("gis__raster_data")
dtype

Within the BMI, functions that describe the grids that variables are defined on take an index instead of a variable name.

Get the grid index for the raster data variable.

In [ ]:
grid = m.get_var_grid("gis__raster_data")
grid

Then find the total size of the raster data.

In [ ]:
size = m.get_grid_size(grid)
size

Next, get the raster data values.

Two notes, however:

* As a rule, memory should not be allocated within a BMI. This leads to the un-Pythonic way that we get the data--first creating an empty array, then passing it to a BMI function to receive values.
* BMI arrays are flattened. This obviates array ordering issues between languages, but it does make >1D data harder to work with.

Allocate an array for the raster data.

In [ ]:
raster = np.ndarray(size, dtype)
raster

Get the data.

In [ ]:
m.get_value("gis__raster_data", raster)

Note that the array is one-dimensional.

In [ ]:
raster.shape

### Reshape the data

Like all BMI arrays, the raster values returned from the BMI `get_value` function are flattened.
Let's restore their original dimensionality.

First, determine the dimensionality of the raster variable.

In [ ]:
rank = m.get_grid_rank(grid)
rank

Get the dimensions of the raster data, first creating an array to store their values.

In [ ]:
shape = np.empty(rank, dtype=int)
m.get_grid_shape(grid, shape)

Reshape the raster data, creating a new array.

In [ ]:
rasterRGB = raster.reshape(shape)

In [ ]:
rasterRGB.shape

### Get map projection information

The data in the GeoTIFF file are georeferenced.
The second and third variables exposed through the BMI,
"gis__coordinate_reference_system_well_known_text" and "gis__affine_transform", respectively,
contain the [WKT](https://en.wikipedia.org/wiki/Well-known_text_representation_of_coordinate_reference_systems) string
and the [affine transform](https://rasterio.readthedocs.io/en/latest/topics/georeferencing.html#affine) for the data.

Get the transform through the BMI, following a process similar to what was used to obtain the raster data:

* get variable type
* get id of grid on which variable is defined
* get size of grid
* use the above to allocate an array for the transform
* get the transform

In [ ]:
dtype = m.get_var_type("gis__affine_transform")
dtype

In [ ]:
grid = m.get_var_grid("gis__affine_transform")
grid

In [ ]:
size = m.get_grid_size(grid)
size

In [ ]:
transform = np.ndarray(size, dtype)

In [ ]:
m.get_value("gis__affine_transform", transform)
transform

## Visualize

Let's visualize the raster data as an image, with a little help from rasterio.

In [ ]:
show(rasterRGB, transform=list(transform[0:6]))

## Conclusion

Last, call the BMI `finalize` function.

In [ ]:
m.finalize()

This demonstration of the BMI took a lot of code to reproduce a simple result.
So why would anyone want to use the BMI?
The key is that, in this demonstration, only the functions belonging to the BMI were used to access the data.
No knowledge of the calling syntax of the underlying `GeoTiff` class was used.

The lesson is: once you've seen one BMI, you've seen them all!